In [2]:
import os
from dotenv import load_dotenv
load_dotenv('keys.env')
openai_api_key = os.getenv('OPENAI_API_KEY')

In [4]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.7)

In [5]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['subject'],
    template = "I want to learn {subject}. What is the first thing I should learn? Answer shortly and accurately."
)

prompt_template_name.format(subject="Python")

'I want to learn Python. What is the first thing I should learn? Answer shortly and accurately.'

In [6]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("SQL")

'\n\nThe first thing to learn when starting to learn SQL is the basic syntax and keywords, such as SELECT, FROM, WHERE, INSERT, UPDATE, and DELETE.'

In [20]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables = ['subject'],
    template = "I want to start a new business for {subject}. Suggest a funny name for the company."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['business_name'],
    template = "Suggest some items for {business_name}. Return it as a comma separated list"
)

items_chain = LLMChain(llm=llm, prompt=prompt_template_items)


In [22]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, items_chain])
response = chain.run("Coffee")
print(response)



Coffee beans, Tea leaves, Energy drinks, Espresso machines, French presses, Coffee grinders, Coffee mugs, Milk frothers, Tea kettles, Coffee filters, Sugar cubes, Creamers, Espresso cups, Coffee thermoses, Tea infusers.


In [32]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables = ['subject'],
    template = "I want to start a new business for {subject}. Suggest a funny name for the company."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key='business_name')

prompt_template_items = PromptTemplate(
    input_variables = ['business_name'],
    template = "Suggest some items for {business_name}. Return it as a comma separated list"
)

items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key='items')

In [34]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, items_chain],
    input_variables = ['subject'],
    output_variables = ['business_name', 'items']
)

chain({'subject': 'Coffee'})

{'subject': 'Coffee',
 'business_name': '\n\nJava Jive!',
 'items': '\n\nCoffee Mugs, Coffee Beans, Espresso Machine, Coffee Grinder, French Press, Coffee Scoop, Sugar Cubes, Creamer, K-Cups, Honey, Cocoa Powder, Tea Infuser, Tea Kettle, Tea Bags, Coffee Filter, Milk Frother'}

In [37]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=8e23ee8be863ec0528aab8c1d137e14744669686870857b9b01508d24cad2ed9
  Stored in directory: /Users/evierangelova/Library/Caches/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [45]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("When was Sam Altman born? What is his age in 2023?")



> Entering new AgentExecutor chain...
 Sam Altman is a person so I should search for him on Wikipedia.
Action: Wikipedia
Action Input: Sam Altman
Observation: Page: Sam Altman
Summary: Samuel Harris Altman ( AWLT-mən; born 1985) is an American entrepreneur, investor, and programmer. He was the co-founder of Loopt and is the current CEO of OpenAI. He was the president of Y Combinator and was briefly the CEO of Reddit.



Page: OpenAI
Summary: OpenAI is an American artificial intelligence (AI) research laboratory consisting of the non-profit OpenAI, Inc. and its for-profit subsidiary corporation OpenAI, L.P.. OpenAI conducts research on artificial intelligence with the declared intention of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work".OpenAI was founded in 2015 by Ilya Sutskever, Greg Brockman, Trevor Blackwell, Vicki Cheung, Andrej Karpathy, Durk Kingma, Jessi

'Sam Altman was born in 1985 and will be 38 years old in 2023.'

In [46]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)
                          

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [48]:
convo.run("Who won the last world cup")

' The last World Cup was won by France in 2018. They defeated Croatia 4–2 in the final.'

In [49]:
convo.run("What is 5+7?")

' 5+7 is equal to 12.'

In [50]:
convo.run("Who was the capitan of the winning team?")

' The captain of the 2018 French World Cup winning team was Hugo Lloris.'

In [53]:
print(convo.memory.buffer)

Human: Who won the world cup in december 2022?
AI:  I'm sorry, I don't know the answer to that question.
Human: Who won the last world cup
AI:  The last World Cup was won by France in 2018. They defeated Croatia 4–2 in the final.
Human: What is 5+7?
AI:  5+7 is equal to 12.
Human: Who was the capitan of the winning team?
AI:  The captain of the 2018 French World Cup winning team was Hugo Lloris.


In [59]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory (k=1)

convo = ConversationChain(
    llm = OpenAI(temperature=0.7),
    memory=memory
)

convo.run("Who won the last world cup?")

' France won the 2018 FIFA World Cup in Russia. They defeated Croatia 4–2 in the final.'

In [60]:
convo.run("What is 5+7?")

' 12.'

In [61]:
convo.run("Who was the capitan of the winning team?")

" I'm sorry, I don't know."